In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.data import DataType
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineEndpoint
import os

In [ ]:
ws = Workspace.from_config()

In [ ]:
cluster_name = 'automlimagescompute'
# compute_target = ws.compute_targets[cluster_name]
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6',
        idle_seconds_before_scaledown=120,
        min_nodes=0,
        max_nodes=3,
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min_node_count is provided, it will use the scale settings for the cluster.
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20
    )
    

In [ ]:
default_ds = ws.get_default_datastore()

In [ ]:
new_container_name = 'streetimagestore'
default_ds.blob_service.create_container(new_container_name)

try:
    imagestore = Datastore.get(ws, new_container_name)
    print("Found Blob Datastore with name: %s" % new_container_name)
except Exception:
    imagestore = Datastore.register_azure_blob_container(
     workspace=ws,
        datastore_name = new_container_name, 
        account_name = default_ds.account_name,
        container_name = new_container_name,
        account_key = default_ds.account_key)
    print("Registered blob datastore with name: %s" % new_container_name)

# Upload images to new datastore
imagestore.upload('./sample_images', 'aml_images', overwrite=True, show_progress=True)

In [ ]:
default_ds.upload('./aml_annotations', 'labeled_image_files', overwrite=True, show_progress=True)

train_image_dataset = Dataset.Tabular.from_json_lines_files(path=(default_ds, 'labeled_image_files/labeled_images_train.jsonl'), set_column_types={"image_url": DataType.to_stream(imagestore.workspace)})
train_image_dataset.register(ws, 'TRAIN_AML_Labeled_Street_Images', create_new_version=True)

test_image_dataset = Dataset.Tabular.from_json_lines_files(path=(default_ds, 'labeled_image_files/labeled_images_test.jsonl'), set_column_types={"image_url": DataType.to_stream(imagestore.workspace)})
test_image_dataset.register(ws, 'TEST_AML_Labeled_Street_Images', create_new_version=True)